In [1]:
import mxnet as mx
import numpy as np
import h5py
import logging
logging.getLogger().setLevel(logging.INFO)

dataset = h5py.File('datasets/datasets.h5', 'r')
train_set_x_orig = mx.nd.array(dataset['train_set_x'][:])
train_set_y_orig = mx.nd.array(dataset['train_set_y'][:])
test_set_x_orig = mx.nd.array(dataset['test_set_x'][:])
test_set_y_orig = mx.nd.array(dataset['test_set_y'][:])

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
train_set_x_orig.shape

(209, 64, 64, 3)

In [3]:
def transform(x):
    v = x.reshape((x.shape[0], (x.shape[1] * x.shape[2]) * x.shape[3]))
    return v.astype(np.float32) / 255

In [4]:
data = transform(train_set_x_orig)
label = train_set_y_orig.astype(np.float32)
test_data = transform(test_set_x_orig)
test_label = test_set_y_orig.astype(np.float32)

In [5]:
data.shape

(209, 12288)

In [6]:
batch_size = 209
trainer_iter = mx.io.NDArrayIter(data, label, batch_size=batch_size, shuffle=False)
val_iter = mx.io.NDArrayIter(test_data, test_label)
#trainer_iter = mx.io.NDArrayIter(np.random.randn(209, 12288), label=np.random.randn(209,), batch_size=209)
#val_iter = mx.io.NDArrayIter(np.zeros((50, 12288)), label=np.zeros((50,)))

In [7]:
data = mx.sym.var('data')
#data = mx.sym.flatten(data=data)
fc1 = mx.sym.FullyConnected(data=data, num_hidden=20)
act1 = mx.sym.Activation(data=fc1, act_type='relu')
fc2 = mx.sym.FullyConnected(data=act1, num_hidden=7)
act2 = mx.sym.Activation(data=fc2, act_type='relu')
fc3 = mx.sym.FullyConnected(data=act2, num_hidden=5)
act3 = mx.sym.Activation(data=fc3, act_type="relu")
fc4 = mx.sym.FullyConnected(data=act3, num_hidden=1)
#act4 = mx.sym.Activation(data=fc4, act_type='sigmoid')
out = mx.sym.LogisticRegressionOutput(data=fc4, name='softmax')
#mx.viz.plot_network(mlp)

__Intermediate level API__


In [8]:
mod = mx.mod.Module(symbol=out,
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['softmax_label']
                   )
# allocate memory given the input data and label shapes
mod.bind(data_shapes=trainer_iter.provide_data, label_shapes=trainer_iter.provide_label)
# initialize parameters by uniform random numbers
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))
# use SGD with learning rate 0.1 to train
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.0075), ))
# use accuracy as the metric
accuracy = mx.metric.Accuracy()
ce_loss = mx.metric.CrossEntropy()
#metric = mx.metric.CompositeEvalMetric([accuracy, ce_loss])
metric = mx.metric.Loss()
# train 5 epochs, i.e. going over the data iter one pass
for epoch in range(3000):
    trainer_iter.reset()
    metric.reset()
    for batch in trainer_iter:
        mod.forward(batch, is_train=True)       # compute predictions
        mod.update_metric(metric, batch.label)  # accumulate prediction accuracy
        mod.backward()                          # compute gradients
        mod.update()                            # update parameters
    if epoch % 100 ==0:
        print('Epoch %d, Training %s' % (epoch, metric.get()))


Epoch 0, Training ('loss', 0.53440561248925311)
Epoch 100, Training ('loss', 0.48359541459517047)
Epoch 200, Training ('loss', 0.47309787878009119)
Epoch 300, Training ('loss', 0.47821694698059958)
Epoch 400, Training ('loss', 0.49147621410315118)
Epoch 500, Training ('loss', 0.481479918557491)
Epoch 600, Training ('loss', 0.48924215216385691)
Epoch 700, Training ('loss', 0.48965468703274523)
Epoch 800, Training ('loss', 0.49024375659997382)
Epoch 900, Training ('loss', 0.48392727500513982)
Epoch 1000, Training ('loss', 0.48117875824704692)
Epoch 1100, Training ('loss', 0.48130002546538575)
Epoch 1200, Training ('loss', 0.47072101209722639)
Epoch 1300, Training ('loss', 0.46221438321200287)
Epoch 1400, Training ('loss', 0.45168906763980265)
Epoch 1500, Training ('loss', 0.44834604217675311)
Epoch 1600, Training ('loss', 0.44284550424968228)
Epoch 1700, Training ('loss', 0.43665594803659541)
Epoch 1800, Training ('loss', 0.42857251327003587)
Epoch 1900, Training ('loss', 0.4256648743551

In [9]:
mod.score(val_iter, mx.metric.Accuracy())

[('accuracy', 0.34000000000000002)]

---

```python
mlp_model = mx.mod.Module(symbol=out, context=mx.cpu())
mlp_model.fit(
    trainer_iter,
    eval_data=val_iter,
    optimizer='sgd',
    optimizer_params={'learning_rate':0.0075},
    eval_metric='loss',
    batch_end_callback = mx.callback.Speedometer(batch_size, 100),
    num_epoch=10
)
```